# Value iteration

![](imgs/value_iteration_1.png)

![](imgs/value_iteration_2.png)

![](imgs/value_iteration_3.png)

![](imgs/value_iteration_4.png)

![](imgs/value_iteration_5.png)

![](imgs/value_iteration_6.png)

In [10]:
!pip install gymnasium

In [11]:
import random
import numpy as np
import gymnasium as gym
import time
from gymnasium import spaces
import os


# custom 2d grid world enviroment
class GridWorld(gym.Env):
    metadata = {'render.modes': ['console']}

    # actions available
    UP = 0
    LEFT = 1
    DOWN = 2
    RIGHT = 3


    def __init__(self, width, height):
        super(GridWorld, self).__init__()
        self.ACTION_NAMES = ["UP", "LEFT", "DOWN", "RIGHT"]
        self.num_actions = 4

        self.size = width * height  # size of the grid world
        self.num_states = self.size
        self.width = width
        self.height = height
        self.num_obstacles = int((width+height)/2)
        self.end_state = np.array([height - 1, width - 1], dtype=np.uint8) # goal state = bottom right cell

        # actions of agents : up, down, left and right
        self.action_space = spaces.Discrete(4)
        # observation : cell indices in the grid
        self.observation_space = spaces.MultiDiscrete([self.height, self.width])

        self.obstacles = np.zeros((height, width))

        for i in range(self.num_obstacles):
            self.obstacles[ random.randrange(height) , random.randrange(width)] = 1

        self.num_steps = 0
        self.max_steps = height*width

        self.current_state = np.zeros((2), np.uint8)#init state = [0,0]

        self.directions = np.array([
            [-1,0], #UP
            [0,-1], #LEFT
            [1,0], #DOWN
            [0,1] #RIGHT
        ])

    def transition_function(self, s, a):
        s_prime = s + self.directions[a,:]

        if s_prime[0] < self.height and s_prime[1] < self.width and (s_prime >= 0).all():
            if self.obstacles[s_prime[0], s_prime[1]] == 0 :
                return s_prime

        return s

    def transition_probabilities(self, s, a):
        prob_next_state = np.zeros((self.heigth, self.width))
        s_prime = self.transition_function(s, a)

        # in each position you will have probability 0
        # in position s', the next state, found with the transition function
        # you will have 1
        # it means that this is a deterministic grid world
        prob_next_state[s_prime[0], s_prime[1]] = 1.0

        return prob_next_state #.flatten()

    def reward_function(self,s):
        r = 0
        if (s == self.end_state).all():
            r = 1

        return r

    def termination_condition(self, s):

        terminated = (s == self.end_state).all()
        truncated = self.num_steps > self.max_steps

        return terminated, truncated

    def step(self, action):
        s_prime = self.transition_function(self.current_state, action)
        reward = self.reward_function(s_prime)
        terminated, truncated = self.termination_condition(s_prime)

        self.current_state = s_prime
        self.num_steps += 1

        return self.current_state, reward, terminated, truncated, None

    def render(self):
        '''
            render the state
        '''

        row = self.current_state[0]
        col = self.current_state[1]

        for r in range(self.height):
            for c in range(self.width):
                if r == row and c == col:
                    print("| A ", end='')
                elif r == self.end_state[0] and c == self.end_state[1]:
                    print("| G ", end='')
                else:
                    if self.obstacles[r,c] == 1:
                        print('|///', end='')
                    else:
                        print('|___', end='')
            print('|')
        print('\n')



    def reset(self):
        self.current_state = np.zeros((2), np.uint8)
        self.num_steps = 0
        return self.current_state


    # computes the map of the rewards: a table
    # that associate the reward to each state of the grid world
    def reward_probabilities(self):
        rewards = np.zeros((self.num_states))
        i = 0
        for r in range(self.height):
            for c in range(self.width):
                state = np.array([r,c], dtype=np.uint8)
                rewards[i] = self.reward_function(state)
                i+=1

        return rewards

    def close(self):
        pass


class NonDeterministicGridWorld(GridWorld):
    def __init__(self, width, height, p=0.8):
        super(NonDeterministicGridWorld, self).__init__(width, height)
        self.probability_right_action = p

    def transition_function(self, s, a):
        s_prime = s + self.directions[a, :]

        #with probability 1 - p diagonal movement
        if random.random() <= 1 - self.probability_right_action:
            if random.random() < 0.5:
                s_prime = s_prime + self.directions[(a+1)%self.num_actions, :]
            else:
                s_prime = s_prime + self.directions[(a-1)%self.num_actions, :]


        if s_prime[0] < self.height and s_prime[1] < self.width and (s_prime >= 0).all():
            if self.obstacles[s_prime[0], s_prime[1]] == 0 :
                return s_prime

        return s

    def transition_probabilities(self, s, a):
        cells = []
        probs = []

        # this associate to each state the probability of ending up there from s appling
        # action a. This time, since the grid world is non-deterministic
        # you won't want just 0 and 1
        prob_next_state = np.zeros((self.height, self.width))

        # right in the sense of "correct"
        # this will insert the probability of ending up in the intended cell inside the prob_next_state matrix
        s_prime_right =  s + self.directions[a, :]
        if s_prime_right[0] < self.height and s_prime_right[1] < self.width and (s_prime_right >= 0).all():
            if self.obstacles[s_prime_right[0], s_prime_right[1]] == 0:
                prob_next_state[s_prime_right[0], s_prime_right[1]] = self.probability_right_action
                cells.append(s_prime_right)
                probs.append(self.probability_right_action)

        # these two above are the probability of ending up in the random diagonally-chosen cell
        s_prime = s_prime_right + self.directions[(a + 1) % self.num_actions, :]
        if s_prime[0] < self.height and s_prime[1] < self.width and (s_prime >= 0).all():
            if self.obstacles[s_prime[0], s_prime[1]] == 0 :
                prob_next_state[s_prime[0], s_prime[1]] = (1 - self.probability_right_action) / 2
                cells.append(s_prime.copy())
                probs.append((1 - self.probability_right_action) / 2)

        s_prime = s_prime_right + self.directions[(a - 1) % self.num_actions, :]
        if s_prime[0] < self.height and s_prime[1] < self.width and (s_prime >= 0).all():
            if self.obstacles[s_prime[0], s_prime[1]] == 0 :
                prob_next_state[s_prime[0], s_prime[1]] = (1 - self.probability_right_action) / 2
                cells.append(s_prime.copy())
                probs.append((1 - self.probability_right_action) / 2)

        #normalization
        sump = sum(probs)
        #for cell in cells:
        #    prob_next_state[cell[0], cell[1]] /= sump
        prob_next_state[s[0], s[1]] = 1 - sump
        return prob_next_state


To apply value iteration we need the **transition probabilities** and the **reward function**

Print the probability over the next state

In [12]:
env = NonDeterministicGridWorld(3,5)
state = env.reset()
env.render()
# next state if we start from state 0,0 and we do action down
next_state_prob = env.transition_probabilities(state, 2)
print(next_state_prob)

| A |///|___|
|___|___|___|
|___|___|///|
|___|___|___|
|___|///| G |


[[0.1 0.  0. ]
 [0.8 0.1 0. ]
 [0.  0.  0. ]
 [0.  0.  0. ]
 [0.  0.  0. ]]


reward values

In [13]:
print(env.reward_probabilities())


[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]


# Value iteration algorithm

![](imgs/value_iteration.png)

In [14]:
def value_iteration(env, gamma=0.99, iters=100):
    # initialize values
    values = np.zeros((env.num_states))
    best_actions = np.zeros((env.num_states), dtype=int)

    # this is a 15 x 2, because each state has two coordinates
    STATES = np.zeros((env.num_states, 2), dtype=np.uint8)
    REWARDS = env.reward_probabilities()
    print(REWARDS)
    i = 0
    for r in range(env.height):
        for c in range(env.width):
            state = np.array([r, c], dtype=np.uint8)
            STATES[i] = state
            i += 1
    delta = 1
    for i in range(iters):
        v_old = values.copy()

        # for each state
        for s in range(env.num_states):
            state = STATES[s]

            if (state == env.end_state).all() or i >= env.max_steps:
                continue # if we reach the termination condition, we cannot perform any action


            max_va = -np.inf
            best_a = 0

            # for the current state s, compute the best action as the one that maximizes v
            # and update values[s] and best_action[s]
            for a in range(env.num_actions):
                next_state_prob = env.transition_probabilities(state, a).flatten()

                # Bellman equation for v
                va = (next_state_prob*(REWARDS + gamma*v_old)).sum()

                if va > max_va:
                    max_va = va
                    best_a = a
            values[s] = max_va
            best_actions[s] = best_a


    # finally, having updated the best action and its value v for each state s, we return the "map"
    return values.reshape((env.height, env.width)), best_actions.reshape((env.height, env.width))

estimate values

In [15]:
values, best_actions = value_iteration(env)

print(values)
print(best_actions)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[[0.94830519 0.95441123 0.94676876]
 [0.95809921 0.96575885 0.95635226]
 [0.96774609 0.97770444 0.98537072]
 [0.97574406 0.98781902 0.99750623]
 [0.96624123 0.99750623 0.        ]]
[[2 2 2]
 [2 2 1]
 [3 2 2]
 [3 3 2]
 [0 3 0]]


simulate optimal policy


In [16]:
done = False
state = env.reset()

# once you have the best action to perform for each
# state you are done: just always choose the best
# action to perform in the state you are currently
# in and finally you will reach the goal
while not done:
    action = best_actions[state[0],state[1]]

    state, reward, terminated, truncated, _ = env.step(action)
    done = terminated or truncated
    env.render()


|___|///|___|
| A |___|___|
|___|___|///|
|___|___|___|
|___|///| G |


|___|///|___|
|___|___|___|
| A |___|///|
|___|___|___|
|___|///| G |


|___|///|___|
|___| A |___|
|___|___|///|
|___|___|___|
|___|///| G |


|___|///|___|
|___|___|___|
|___| A |///|
|___|___|___|
|___|///| G |


|___|///|___|
|___|___|___|
|___|___|///|
|___| A |___|
|___|///| G |


|___|///|___|
|___|___|___|
|___|___|///|
|___|___| A |
|___|///| G |


|___|///|___|
|___|___|___|
|___|___|///|
|___|___|___|
|___|///| A |


